## Brief Documentation

#### `generate_schema(dataframe, columns) -> str`
Builds a JSON string describing each column’s name, data type, and up to five sample values.

---

#### `execute_pandas_query(dataframe, query_str) -> Any`
Safely evaluates a Python expression on a copy of the provided DataFrame. Returns the outcome.

---

#### `handle_tool_call(tool_call, dataframe) -> Any`
Extracts the function name and arguments from a tool call, then executes the corresponding function.

---

#### `nl_to_query(user_request) -> Any`
Sends the natural-language request to the LLM, which returns a tool call. The call is executed to filter the DataFrame. The function prints and returns the result.


In [2]:
import json
import pandas as pd
from openai import OpenAI

# Load CSVs
df_master = pd.read_csv(r'C:\Users\AakashAI\Desktop\Repositories\Sales Agent\merged_leads.csv')
df_cleaned_leads = pd.read_csv(r'C:\Users\AakashAI\Desktop\Repositories\Sales Agent\cleaned_leads.csv')

client = OpenAI()

FILTER_COLUMNS = [
    "Lead Number", "Lead Source", "Lead Origin", "Do Not Email",
    "Do Not Call", "Converted", "Country", "Lead Stage", "City"
]

In [11]:
def generate_schema(dataframe: pd.DataFrame, columns: list) -> str:
    """
    Returns a JSON string describing the requested columns of a DataFrame:
    column name, data type, and up to five sample values.
    """
    schema_list = []

    for col in columns:
        if col in dataframe.columns:
            schema_list.append({
                "column_name": col,
                "dtype": str(dataframe[col].dtype),
                "examples": dataframe[col].dropna().unique()[:5].tolist()
            })

    return json.dumps(schema_list, indent=2)

def execute_pandas_query(dataframe: pd.DataFrame, query_str: str):
    """
    Evaluates a Python expression on a copy of the given DataFrame.
    Returns either a DataFrame, Series, or any resulting object.
    """
    local_vars = {"df": dataframe.copy()}

    try:
        return eval(query_str, {"__builtins__": {}}, local_vars)
    except Exception as e:
        print(f"Error: {e}")
        return None

def handle_tool_call(tool_call, dataframe: pd.DataFrame):
    """
    Handles the tool call by extracting the function name and arguments,
    running the appropriate function, and returning the result.
    """
    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)

    if function_name == "execute_pandas_query":
        query_str = arguments["query_str"]
        result = execute_pandas_query(dataframe, query_str)
        
        if isinstance(result, pd.DataFrame):
            #Returns filtered data based on the query string in JSON format
            return result.to_dict(orient="records")
        elif isinstance(result, pd.Series):
            return result.tolist()
        
        return result

In [12]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "execute_pandas_query",
            "description": "Run a pandas query string on a DataFrame. Use 'df' as the variable name.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query_str": {
                        "type": "string",
                        "description": "Python code string to evaluate on the DataFrame. Use 'df' as the variable name."
                    }
                },
                "required": ["query_str"]
            }
        }
    }
]

In [ ]:

def nl_to_query(user_request: str):
    """
    Sends the natural language request to the LLM, which determines how to filter
    the DataFrame by calling the 'execute_pandas_query' tool. Returns the final
    filter results in Python object form. If the generated query is invalid, the 
    function retries up to 3 times.
    """
    max_retries = 3
    attempt = 0
    schema_description = generate_schema(df_cleaned_leads, FILTER_COLUMNS)

    while attempt < max_retries:
        print("Entered NL query:", user_request)
        prompt = f"""
            You are an expert data analyst.

            You have access to a tool called `execute_pandas_query` that accepts a Python expression as input and runs it on a DataFrame called `df`.

            Here is the DataFrame schema:
            {schema_description}

            Below are a few one-shot examples of valid filtering queries:

            Example 1:
            User Request: "List all leads from City 'Metropolis' that have not been converted"
            Expected tool call: execute_pandas_query("df[(df['City'] == 'Metropolis') & (df['Converted'] == 0)]")

            Example 2:
            User Request: "Show me leads from Source 'Web' with a lead score above 100"
            Expected tool call: execute_pandas_query("df[(df['Lead Source'] == 'Web') & (df['Lead Score'] > 100)]")

            Example 3:
            User Request: "Find leads that are qualified and in the age group '30-35'"
            Expected tool call: execute_pandas_query("df[(df['Lead Stage'] == 'Qualified') & (df['Age'] == '30-35')]")


            Now, a user has asked to filter the DataFrame with this request:
            '{user_request}'

            You should call the tool with the appropriate pandas filtering expression using `df`.
            Understand the schema and the valid values in the columns.
            rephrase the user query with available values within the particular column.
            Strictly align with the query and do not give any other information or irrelevant details.
            Only call the tool. Do not answer the query yourself.
        """

        messages = [
            {"role": "system", "content": "You are a pandas expert."},
            {"role": "user", "content": prompt}
        ]

        response = client.chat.completions.create(
            messages=messages,
            model="gpt-4o",
            stream=False,
            tool_choice="auto",
            tools=tools,
            temperature=0
        )

        try:
            response_content = response.choices[0].message.content
            if response_content:
                print(response_content.strip())
            else:
                print("No direct LLM response. Possibly a tool call only.")
        except Exception as e:
            print(f"Error extracting LLM response: {e}")

        # Extract the first tool call
        try:
            tool_call = response.choices[0].message.tool_calls[0]
            print(f"Tool call: {tool_call}")
        except Exception as e:
            print("Failed to extract a tool call from the LLM response:", e)
            # If the LLM response is empty or invalid, retry with pandas query generation

            attempt += 1
            continue  # Retry the LLM call

        # Validate query by attempting to execute it
        try:
            tool_result = handle_tool_call(tool_call, df_cleaned_leads)
            print(tool_result)
            return tool_result  # Return if successful
        except Exception as e:
            print(f"Query execution failed on attempt {attempt+1}: {e}")
            attempt += 1
            print("Retrying query generation...\n")

    # If all attempts fail, raise an exception
    raise Exception("Failed to generate a valid query after maximum retry attempts.")

# Example usage:
nl_query = "Can you get me leads Named yogesh?"
result = nl_to_query(nl_query)


Entered NL query: Can you get me leads Named yogesh?
No direct LLM response. Possibly a tool call only.
Tool call: ChatCompletionMessageToolCall(id='call_rnlej1QiyFJ7oAdVHnBHJsHf', function=Function(arguments='{"query_str":"df[df[\'Lead Source\'] == \'Reference\']"}', name='execute_pandas_query'), type='function')
[{'Lead Number': 651812, 'Lead Source': 'Reference', 'Lead Origin': 'Lead Add Form', 'Do Not Email': 'No', 'Do Not Call': 'No', 'Converted': 1, 'Country': nan, 'Mobile Number': nan, 'Lead Stage': 'Closed', 'Lead Grade': 'B', 'Lead Score': 325, 'Age': 'Select', 'Email': 'nehamenon26@rediffmail.com', 'City': 'Select'}, {'Lead Number': 648284, 'Lead Source': 'Reference', 'Lead Origin': 'Lead Add Form', 'Do Not Email': 'No', 'Do Not Call': 'No', 'Converted': 1, 'Country': nan, 'Mobile Number': nan, 'Lead Stage': 'Qualified', 'Lead Grade': nan, 'Lead Score': 0, 'Age': 'Select', 'Email': 'manish.chand97@Yahoo.com', 'City': 'Mumbai'}, {'Lead Number': 639288, 'Lead Source': 'Referenc

In [32]:
result

[]

### Evaluation

In [38]:
test_cases = [
    {
        "id": 1,
        "description": "Leads from Mumbai who haven't converted",
        "nl_query": "Get me leads from Mumbai who haven’t converted",
        "expected_code": "df[(df['City'] == 'Mumbai') & (df['Converted'] == 0)]",
        "expected_lead_numbers": [659357, 655287, 615883],
        "expected_count": 3
    },
    {
        "id": 2,
        "description": "Direct Traffic leads who are not interested",
        "nl_query": "Show me leads from Direct Traffic that are not interested",
        "expected_code": "df[(df['Lead Source'] == 'Direct Traffic') & (df['Lead Stage'] == 'Not Interested')]",
        "expected_lead_numbers": [655287, 625880],
        "expected_count": 2
    },
    {
        "id": 3,
        "description": "Leads from Olark Chat who converted",
        "nl_query": "Find all leads from Olark Chat who converted",
        "expected_code": "df[(df['Lead Source'] == 'Olark Chat') & (df['Converted'] == 1)]",
        "expected_lead_numbers": [653724, 646302, 635995, 631411, 624762, 617610, 616058],
        "expected_count": 7
    },
    {
        "id": 4,
        "description": "Leads with Do Not Email or Do Not Call = 'Yes'",
        "nl_query": "Which leads have 'Do Not Email' or 'Do Not Call' as Yes?",
        "expected_code": "df[(df['Do Not Email'] == 'Yes') | (df['Do Not Call'] == 'Yes')]",
        "expected_lead_numbers": [646302, 595717],
        "expected_count": 2
    },
    {
        "id": 5,
        "description": "Qualified leads in Mumbai with Lead Score > 80",
        "nl_query": "Get me qualified leads in Mumbai whose lead score is above 80",
        "expected_code": "df[(df['City'] == 'Mumbai') & (df['Lead Stage'] == 'Qualified') & (df['Lead Score'] > 80)]",
        "expected_lead_numbers": [637980, 617845, 613622, 605652, 588124],
        "expected_count": 5
    },
    {
        "id": 6,
        "description": "Leads in age group 21-25",
        "nl_query": "Show me all leads between the age of 21 and 25",
        "expected_code": "df[df['Age'] == '21-25']",
        "expected_lead_numbers": [636404, 617845],
        "expected_count": 2
    },
    {
        "id": 7,
        "description": "Number of leads with API origin",
        "nl_query": "How many leads came from an API origin?",
        "expected_code": "len(df[df['Lead Origin'] == 'API'])",
        "expected_count": 12
    },
    {
        "id": 8,
        "description": "Leads from India with Lead Grade B",
        "nl_query": "Can you get me leads from India who have a lead grade of B?",
        "expected_code": "df[(df['Country'] == 'India') & (df['Lead Grade'] == 'B')]",
        "expected_lead_numbers": [654061, 649941, 636404, 617845, 616761],
        "expected_count": 5
    },
    {
        "id": 9,
        "description": "Closed leads with Lead Score above 300",
        "nl_query": "Get me all closed leads that have a lead score more than 300",
        "expected_code": "df[(df['Lead Stage'] == 'Closed') & (df['Lead Score'] > 300)]",
        "expected_lead_numbers": [651812, 643051, 615524],
        "expected_count": 3
    },
    {
        "id": 10,
        "description": "Unreachable leads from Organic Search",
        "nl_query": "List all unreachable leads that came from Organic Search",
        "expected_code": "df[(df['Lead Stage'] == 'Unreachable') & (df['Lead Source'] == 'Organic Search')]",
        "expected_lead_numbers": [660030],
        "expected_count": 1
    }
]

import ast

def accuracy_tester(df, test_cases):
    """
    Runs each test case against your 'nl_to_query' system:
      1) Calls the system with the natural-language prompt.
      2) Evaluates the returned code/logic on df.
      3) Compares with the expected results.
    Prints pass/fail summaries and returns overall accuracy.
    """
    passed = 0
    total_tests = len(test_cases)

    for test in test_cases:
        print(f"--- Test Case {test['id']}: {test['description']} ---")

        # 1) Get final data from your pipeline function
        #    (this is presumably the 'tool_result' from handle_tool_call)
        tool_result = nl_to_query(test["nl_query"])

        # 2) Parse the returned data (list of dicts, int, etc.)
        if isinstance(tool_result, list):
            # e.g. a list of row-dictionaries
            actual_count = len(tool_result)
            # If we need to compare lead numbers, gather them
            actual_leads = sorted(
                row["Lead Number"] for row in tool_result if "Lead Number" in row
            )
        elif isinstance(tool_result, int):
            # If the tool directly returned a count
            actual_count = tool_result
            actual_leads = []
        else:
            # Could be None or something else. Adapt as needed.
            actual_count = 0
            actual_leads = []

        # 3) Compare to expected_count and/or expected_lead_numbers
        expected_count = test.get("expected_count")
        expected_leads = test.get("expected_lead_numbers", [])

        # If a test doesn't specify a count, we won't enforce it
        if expected_count is not None:
            is_count_correct = (actual_count == expected_count)
        else:
            is_count_correct = True

        # If a test doesn't specify lead numbers, skip that check
        if expected_leads:
            is_leads_correct = (actual_leads == sorted(expected_leads))
        else:
            is_leads_correct = True

        # 4) Decide pass/fail
        if is_count_correct and is_leads_correct:
            passed += 1
            print("  PASS")
        else:
            print("  FAIL")
            print(f"    Expected Count: {expected_count}, Actual Count: {actual_count}")
            print(f"    Expected Leads: {expected_leads}, Actual Leads: {actual_leads}")

    accuracy = passed / total_tests * 100
    print(f"\nTotal Passed: {passed}/{total_tests} tests. Accuracy: {accuracy:.1f}%")
    return accuracy


In [39]:
import json

json_test_data = {
    "test_cases": test_cases
}

with open("test_data.json", "w") as f:
    json.dump(json_test_data, f, indent=2)

with open("test_data.json", "r") as f:
    data = json.load(f)

test_cases = data["test_cases"]
accuracy_tester(df_cleaned_leads, test_cases)


--- Test Case 1: Leads from Mumbai who haven't converted ---
Entered NL query: Get me leads from Mumbai who haven’t converted
No direct LLM response. Possibly a tool call only.
Tool call: ChatCompletionMessageToolCall(id='call_lsX5iQ3ax551IFdTpB902BK5', function=Function(arguments='{"query_str":"df[(df[\'City\'] == \'Mumbai\') & (df[\'Converted\'] == 0)]"}', name='execute_pandas_query'), type='function')
[{'Lead Number': 659357, 'Lead Source': 'Google', 'Lead Origin': 'Landing Page Submission', 'Do Not Email': 'No', 'Do Not Call': 'No', 'Converted': 0, 'Country': 'India', 'Mobile Number': nan, 'Lead Stage': 'Unreachable', 'Lead Grade': nan, 'Lead Score': 120, 'Age': 'Select', 'Email': 'Yogeshsadarang@yahoo.in', 'City': 'Mumbai'}, {'Lead Number': 655287, 'Lead Source': 'Direct Traffic', 'Lead Origin': 'Landing Page Submission', 'Do Not Email': 'No', 'Do Not Call': 'No', 'Converted': 0, 'Country': 'India', 'Mobile Number': nan, 'Lead Stage': 'Not Interested', 'Lead Grade': nan, 'Lead Sco

80.0